# LightFM quick start

### Comment

Two unexpected features:
- Train and Test division in a rather non-homogeneous way for movielens dataset leaving to a variation on performance
- The comparison for one example considers 3 examples on train, wich excludes many possibilities for comparison

In [39]:
# Using MAC or Windows will give you the following warning
import numpy as np
from lightfm import LightFM
from lightfm.datasets import fetch_movielens
from lightfm.evaluation import precision_at_k, recall_at_k,auc_score

In [2]:
# Load the movielens 100k dataset. Only
# five star ratings are treated as positive.
data = fetch_movielens(min_rating = 5.0)

In [4]:
print(repr(data['train']))
print(repr(data['test']))

<943x1682 sparse matrix of type '<class 'numpy.int32'>'
	with 19048 stored elements in COOrdinate format>
<943x1682 sparse matrix of type '<class 'numpy.int32'>'
	with 2153 stored elements in COOrdinate format>


WARP (Weighted Approximate-Rank Pairwise) model

Model training is accomplished via SGD (stochastic gradient descent)

In [6]:
# Instantiate and train the model
model = LightFM(loss= 'warp')
# epochs is the number of passings through data
%time model.fit(data['train'], epochs=30, num_threads=1)

Wall time: 1.83 s


In [42]:
# Evaluate the trained model
print("Train precision: %.2f" % precision_at_k(model, data['train'], k=10).mean())
print("Test precision: %.2f" % precision_at_k(model, data['test'],data['train'], k=10).mean())
print()
#recall_at_k
print("Train recall: %.2f" % recall_at_k(model, data['train'], k=10).mean())
print("Test recall: %.2f" % recall_at_k(model, data['test'],data['train'], k=10).mean())
print()
#auc_score
print("Train auc_score: %.2f" % auc_score(model, data['train']).mean())
print("Test auc_score: %.2f" % auc_score(model, data['test'],data['train']).mean())

Train precision: 0.34
Test precision: 0.06

Train recall: 0.31
Test recall: 0.24

Train auc_score: 0.97
Test auc_score: 0.93


In [73]:
def sample_recommendation(model, data, user_ids):


    n_users, n_items = data['train'].shape

    for user_id in user_ids:
        known_positives = data['item_labels'][data['train'].tocsr()[user_id].indices]

        scores = model.predict(user_id, np.arange(n_items))
        top_items = data['item_labels'][np.argsort(-scores)]

        print("User %s" % user_id)
        print("     Known positives:")

        for x in known_positives[:3]:
            print("        %s" % x)

        print("     Recommended:")

        for x in top_items[:10]:
            print("        %s" % x)
            
        print("     %s" % (len([t for t in top_items[:10] if t in known_positives])/10) )

sample_recommendation(model, data, [0])

User 0
     Known positives:
        Toy Story (1995)
        Shanghai Triad (Yao a yao yao dao waipo qiao) (1995)
        Dead Man Walking (1995)
     Recommended:
        Fargo (1996)
        Star Wars (1977)
        Contact (1997)
        Usual Suspects, The (1995)
        Twelve Monkeys (1995)
        Pulp Fiction (1994)
        Godfather, The (1972)
        Monty Python and the Holy Grail (1974)
        Dead Man Walking (1995)
        Princess Bride, The (1987)
     0.8


In [74]:
data['train']

<943x1682 sparse matrix of type '<class 'numpy.float32'>'
	with 19048 stored elements in COOrdinate format>

In [75]:
import numpy as np
19048*np.float32(5).nbytes

76192

In [18]:
type(data['train'])

scipy.sparse.coo.coo_matrix